Import packages:

In [1]:
import numpy as np
import pandas as pd
import re

Read in the data:

In [2]:
# read the data
datapath = "data/university_data/"

cwurData = pd.read_csv(datapath + "/cwurData.csv", header=0)
cwurData = cwurData.rename(columns={'institution': 'university_name'})

shanghaiData = pd.read_csv(datapath + "/shanghaiData.csv", header=0)

timesData = pd.read_csv(datapath + "/timesData.csv", header=0)

In the world_rank columns are also non-integer characters. We get rid of them to cast the column to int64:

In [3]:
#timesData[timesData.world_rank.str.contains('[^0-9]' ,regex = True)]['world_rank'].unique()

#replacing for example '201-250' with mean(201,250)
rangeRanks = timesData[timesData.world_rank.str.contains('[0-9]-[0-9]' ,regex = True)]['world_rank'].unique()
rangeRanksToAvg = {}
for range in rangeRanks:
    parts = range.split('-')
    avg = int(np.mean([int(parts[0]), int(parts[1])]))
    rangeRanksToAvg[range] = str(avg)
timesData = timesData.replace({"world_rank": rangeRanksToAvg})

rangeRanks = shanghaiData[shanghaiData.world_rank.str.contains('[0-9]-[0-9]' ,regex = True)]['world_rank'].unique()
rangeRanksToAvg = {}
for range in rangeRanks:
    parts = range.split('-')
    avg = int(np.mean([int(parts[0]), int(parts[1])]))
    rangeRanksToAvg[range] = str(avg)
shanghaiData = shanghaiData.replace({"world_rank": rangeRanksToAvg})

#replacing for example '=127' with 127
timesData['world_rank'] = timesData['world_rank'].apply(lambda x : re.sub("[^0-9]", "", str(x)))

timesData = timesData.astype({'world_rank': 'int64'})
shanghaiData = shanghaiData.astype({'world_rank': 'int64'})

Same countries are referred differently across the datasets. There are also some typos in these columns. So we try to standardize them:

In [4]:
#timesCountries = timesData['country'].unique()
#cwurCountries = cwurData['country'].unique()

#timesCountriesFiltered = [x for x in timesCountries if x not in cwurCountries]
#cwurCountriesFiltered = [x for x in cwurCountries if x not in timesCountries]

#getting the same country names
countriesDict = {'United States of America':'USA',
                 'Unisted States of America':'USA',
                 'Unted Kingdom':'United Kingdom',
                 'Republic of Ireland':'Ireland',
                 'Russian Federation':'Russia',
                 'Slovakia':'Slovak Republic'}

timesData = timesData.replace({"country": countriesDict})

timesCountries = timesData['country'].unique()
cwurCountries = cwurData['country'].unique()

timesCountriesFiltered = [x for x in timesCountries if x not in cwurCountries]
cwurCountriesFiltered = [x for x in cwurCountries if x not in timesCountries]

#timesCountriesFiltered
#cwurCountriesFiltered

Same universities are referred differently across the datasets. So we try to standardize them:

In [5]:
#timesUnis = timesData['university_name'].unique()
#cwurUnis = cwurData['university_name'].unique()

#timesUnisFiltered = [x for x in timesUnis if x not in cwurUnis]
#cwurUnisFiltered = [x for x in cwurUnis if x not in timesUnis]

#I tried to get the same university_names, but its simply to tedious...
unisDict = {'ETH Zurich – Swiss Federal Institute of Technology Zurich':'Swiss Federal Institute of Technology in Zurich',
            'University of Michigan':'University of Michigan, Ann Arbor',
            'University of Washington':'University of Washington - Seattle',
            'University of Illinois at Urbana-Champaign':'University of Illinois at Urbana–Champaign',
            'Washington University in St Louis':'Washington University in St. Louis',
            'University of Wisconsin':'University of Wisconsin–Madison',
            'University of Minnesota':'University of Minnesota, Twin Cities',
            'Massachusetts Institute of Technology (MIT)':'Massachusetts Institute of Technology',
            'University of Wisconsin - Madison':'University of Wisconsin–Madison',
            'The University of Tokyo':'University of Tokyo',
            'University of Michigan - Ann Arbor':'University of Michigan, Ann Arbor',
            'The Imperial College of Science, Technology and Medicine':'Imperial College London',
            'Swiss Federal Institute of Technology Zurich':'Swiss Federal Institute of Technology in Zurich',
            'University of Colorado at Boulder':'University of Colorado Boulder',
            'The University of Texas at Austin':'University of Texas at Austin',
            'The University of Texas Southwestern Medical Center at Dallas':'University of Texas Southwestern Medical Center',
            'The University of Edinburgh':'University of Edinburgh'}

timesData = timesData.replace({"university_name": unisDict})
shanghaiData = shanghaiData.replace({"university_name": unisDict})

timesUnis = timesData['university_name'].unique()
shanghaiUnis = shanghaiData['university_name'].unique()
cwurUnis = cwurData['university_name'].unique()

timesUnisFiltered = [x for x in timesUnis if x not in cwurUnis]
shanghaiUnisFiltered = [x for x in shanghaiUnis if x not in cwurUnis]
cwurUnisFiltered = [x for x in cwurUnis if x not in shanghaiUnis]
#cwurUnisFiltered = [x for x in cwurUnis if x not in timesUnis]

#timesUnisFiltered
#shanghaiUnisFiltered
#cwurUnisFiltered

The shanghai Dataset does not contain country information. We add it by joining it with the other dataframes:

In [6]:
#get countries of universities for shanghaiData
uniCountryDf = pd.concat([timesData[['university_name', 'country']], cwurData[['university_name', 'country']]]).drop_duplicates()
shanghaiData = pd.merge(shanghaiData, uniCountryDf, how='left', on='university_name')